In [2]:
pip install pandas

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/341.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/341.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/341.8 kB ? eta -:--:--
     --- --------------------------------- 30.7/341.8 kB 145.2 kB/s eta 0:00:03
     --- --------------------------------- 30.7/341.8 kB 145.2 kB/s eta 0:00:03
     --------------- -------------------- 143.4/341.8 kB 607.9 kB/s eta 0:00:01
     --------------- -------------------- 143.4/341.8 kB 607.9 kB/s eta 0:00:01
     ----------------- ------------------ 163.8/341.8 kB 490.7 kB/s eta 0:00:01
     ----

In [3]:
pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
     - -------------------------------------- 0.0/1.5 MB 393.8 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/1.5 MB 573.4 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 573.4 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 573.4 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 573.4 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 573.4 kB/s eta 0:00:03
     ---- ----------------------------------- 0.2/1.5 MB 499.5 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 535.8 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 497.6 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 474.7 kB/s eta 0:00:03
     ------ --------------------------------- 0.2/1

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# Step 1: Load the Dataset
dataset_path = 'HateSpeechDatasetBalanced.csv'
data = pd.read_csv(dataset_path)

# Display the first few rows of the dataset
print("Dataset Sample:")
print(data.head())

Dataset Sample:
                                             Content  Label
0  denial of normal the con be asked to comment o...      1
1  just by being able to tweet this insufferable ...      1
2  that is retarded you too cute to be single tha...      1
3  thought of a real badass mongol style declarat...      1
4                                afro american basho      1


In [2]:
print(data.tail)

<bound method NDFrame.tail of                                                   Content  Label
0       denial of normal the con be asked to comment o...      1
1       just by being able to tweet this insufferable ...      1
2       that is retarded you too cute to be single tha...      1
3       thought of a real badass mongol style declarat...      1
4                                     afro american basho      1
...                                                   ...    ...
726114  i mute this telecasting and played kanye west ...      1
726115  but hell yeah he s not a bachelor but looooooo...      1
726116  great video musician but s not my musician lol...      1
726117  not great pop video yeah he s not a pedophile ...      1
726118  great video yeah he s non a paedophile lolllll...      1

[726119 rows x 2 columns]>


In [3]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())

# Analyze the distribution of labels
print("\nLabel Distribution:")
print(data['Label'].value_counts())


Missing Values:
Content    0
Label      0
dtype: int64

Label Distribution:
Label
1    364525
0    361594
Name: count, dtype: int64


In [4]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adars\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.stem import SnowballStemmer

# Function to perform text preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing special characters, numbers, and stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if (word.isalpha() and word not in stop_words)]

    # Stemming using SnowballStemmer
    stemmer = SnowballStemmer("english")
    tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

data['processed_text'] = data['Content'].apply(preprocess_text)

# Display the preprocessed text
print("\nPreprocessed Text:")
print(data['processed_text'].head())


Preprocessed Text:
0    denial normal con ask comment tragedi emot retard
1    abl tweet insuffer bullshit prove trump nazi v...
2                               retard cute singl life
3    thought real badass mongol style declar war at...
4                                  afro american basho
Name: processed_text, dtype: object


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = data['Content']
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Build and train the Logistic Regression model using TF-IDF features with adjusted max_iter
model_tfidf = LogisticRegression(max_iter=1000)  # Adjust the number of iterations as needed
model_tfidf.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# Evaluate the model
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(f"TF-IDF Model Accuracy: {accuracy_tfidf:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tfidf))

TF-IDF Model Accuracy: 0.82

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.81      0.82     72043
           1       0.82      0.82      0.82     73181

    accuracy                           0.82    145224
   macro avg       0.82      0.82      0.82    145224
weighted avg       0.82      0.82      0.82    145224



In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to perform text preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing special characters, numbers, and stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if (word.isalpha() and word not in stop_words)]

    # Stemming using SnowballStemmer
    stemmer = SnowballStemmer("english")
    tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

# Assuming 'Content' is the column containing preprocessed text, and 'Label' is the column containing labels (0 or 1)
X_train, _, _, _ = train_test_split(data['Content'], data['Label'], test_size=0.2, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Your real input data
real_input_data = ["ok bro messege me when you are done",
                   "you are a pussy. you suck at this game, never play it again",
                   "you are a fucking idiot to think this way"]

# Preprocess the real input data
preprocessed_real_data = [preprocess_text(text) for text in real_input_data]

# Transform the real input data using the fitted vectorizer
real_data_tfidf = tfidf_vectorizer.transform(preprocessed_real_data)

# Make predictions on the real input data
real_data_predictions = model_tfidf.predict(real_data_tfidf)

# Display the predictions
for text, prediction in zip(real_input_data, real_data_predictions):
    print(f"Text: {text}")
    print(f"Predicted Label: {prediction}")
    print("-" * 30)


Text: ok bro messege me when you are done
Predicted Label: 0
------------------------------
Text: you are a pussy. you suck at this game, never play it again
Predicted Label: 1
------------------------------
Text: you are a fucking idiot to think this way
Predicted Label: 1
------------------------------


In [9]:
import joblib
joblib.dump(model_tfidf, 'model_tfidf.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
